In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import time
import csv
import os
from datetime import datetime

# 1. 加载菜单数据
def load_menu_data():
    menu_data = {}
    with open('menu.txt', 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                dish_id = parts[0]
                dish_name = parts[1]
                price = float(parts[2])
                menu_data[dish_id] = (dish_name, price)
    return menu_data

# 2. 初始化Spark环境
sc = SparkContext("local[2]", "DishOrderCalculation")
ssc = StreamingContext(sc, 5)  # 1秒批处理间隔

# 广播菜单数据
menu_broadcast = sc.broadcast(load_menu_data())

# 全局变量
all_orders = []  # 存储所有订单
last_summary_time = time.time()
output_file = None  # 输出文件名
file_counter = 1    # 文件计数器

# 生成新的CSV文件名
def generate_new_filename():
    global file_counter
    # 查找已存在的文件数量
    existing_files = [f for f in os.listdir() if f.startswith("收入汇总-") and f.endswith(".csv")]
    file_counter = len(existing_files) + 1
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    return f"收入汇总-{file_counter}_{timestamp}.csv"

# 初始化CSV文件
def init_csv_file():
    global output_file
    output_file = generate_new_filename()
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["序号", "菜品ID", "菜名", "价格(元)"])
    print(f"已创建新的收入汇总文件: {output_file}")

# 3. 创建DStream
lines = ssc.socketTextStream("localhost", 9999)

# 4. 处理函数
def process_order_batch(rdd):
    global all_orders, last_summary_time
    
    if not rdd.isEmpty():
        current_time = time.time()
        current_batch = []
        
        # 处理当前批次
        orders = rdd.collect()
        
        print("\n=== 新订单明细 ===")
        batch_total = 0.0
        
        for i, dish_id in enumerate(orders, 1):
            menu_info = menu_broadcast.value.get(dish_id, ("未知菜品", 0.0))
            dish_name, price = menu_info
            batch_total += price
            current_batch.append((dish_id, dish_name, price))
            print(f"{i}. 菜品ID: {dish_id}, 菜名: {dish_name}, 价格: {price}元")
        
        print(f"当前批次总价: {batch_total}元")
        
        # 保存到CSV文件
        save_to_csv(current_batch)
        
        # 添加到全局订单列表
        all_orders.extend(current_batch)
        
        # 20秒汇总检查
        if current_time - last_summary_time >= 20:
            print_20sec_summary()
            last_summary_time = current_time

# 保存订单到CSV
def save_to_csv(batch_orders):
    if output_file is None:
        init_csv_file()
    
    with open(output_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for order in batch_orders:
            writer.writerow([len(all_orders), order[0], order[1], order[2]])
    
    # 更新总收入统计
    update_total_summary()

# 更新总收入统计
def update_total_summary():
    total = sum(order[2] for order in all_orders)
    with open(output_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([])  # 空行分隔
        writer.writerow(["总计", "", "", total])
        writer.writerow(["总订单数", len(all_orders)])
        writer.writerow(["生成时间", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])

# 打印20秒汇总
def print_20sec_summary():
    last_20sec = all_orders[-20:] if len(all_orders) > 20 else all_orders
    total = sum(order[2] for order in last_20sec)
    
    print("\n===== 20秒收入汇总 =====")
    print(f"20秒内订单数: {len(last_20sec)}")
    print(f"20秒内总收入: {total}元")
    print("=======================\n")

# 5. 应用处理函数
lines.foreachRDD(process_order_batch)

# 6. 启动流计算
print("正在启动Spark Streaming程序...")
init_csv_file()  # 程序启动时创建新文件
ssc.start()
ssc.awaitTermination()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 20:38:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/root/anaconda3/envs/pyspark/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


正在启动Spark Streaming程序...
已创建新的收入汇总文件: 收入汇总-1_20250511-203825.csv


25/05/11 20:38:32 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/11 20:38:32 WARN BlockManager: Block input-0-1747021112000 replicated to only 0 peer(s) instead of 1 peers
25/05/11 20:38:32 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/11 20:38:32 WARN BlockManager: Block input-0-1747021112600 replicated to only 0 peer(s) instead of 1 peers
25/05/11 20:38:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/11 20:38:33 WARN BlockManager: Block input-0-1747021112800 replicated to only 0 peer(s) instead of 1 peers
25/05/11 20:38:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/11 20:38:33 WARN BlockManager: Block input-0-1747021113000 replicated to only 0 peer(s) instead of 1 peers
25/05/11 20:38:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
25/05/11 20:38:33 WARN BlockManager: Block input-0-1747021113400 replicated to


=== 新订单明细 ===
1. 菜品ID: 1, 菜名: 香菇肥牛, 价格: 58.0元
2. 菜品ID: 1, 菜名: 香菇肥牛, 价格: 58.0元
3. 菜品ID: 2, 菜名: 麻婆豆腐, 价格: 32.0元
4. 菜品ID: 3, 菜名: 红烧茄子, 价格: 15.0元
5. 菜品ID: 4, 菜名: 小炒凉粉, 价格: 16.0元
6. 菜品ID: 5, 菜名: 京酱肉丝, 价格: 22.0元
7. 菜品ID: 1, 菜名: 香菇肥牛, 价格: 58.0元
8. 菜品ID: 2, 菜名: 麻婆豆腐, 价格: 32.0元
9. 菜品ID: 3, 菜名: 红烧茄子, 价格: 15.0元
当前批次总价: 306.0元



=== 新订单明细 ===
1. 菜品ID: 4, 菜名: 小炒凉粉, 价格: 16.0元
2. 菜品ID: 5, 菜名: 京酱肉丝, 价格: 22.0元
当前批次总价: 38.0元


ERROR:root:Exception while sending command.                         (0 + 1) / 1]
Traceback (most recent call last):
  File "/root/anaconda3/envs/pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=68>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/anaconda3/envs/pyspark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/envs/pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call las

Py4JError: An error occurred while calling o25.awaitTermination